In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import numpy as np
import math

import pickle
import os

from torch.utils.data import TensorDataset, DataLoader

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Load pickle file

pickle_file = 'end2end_dataset_v2.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    dev_dataset = save['dev_dataset']
    dev_labels = save['dev_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    
print(train_dataset[0].shape)
print(train_labels.shape)

In [ ]:
# DataLoader
batch_size = 64

train_tensor = TensorDataset(train_dataset[0],train_dataset[1],
                             train_dataset[2],train_dataset[3],train_labels, torch.from_numpy(np.array(train_dataset[4])))
dev_tensor = TensorDataset(dev_dataset[0],dev_dataset[1],
                          dev_dataset[2],dev_dataset[3],dev_labels, torch.from_numpy(np.array(dev_dataset[4])))
test_tensor = TensorDataset(test_dataset[0],test_dataset[1],
                           test_dataset[2],test_dataset[3],test_labels, torch.from_numpy(np.array(test_dataset[4])))
train_loader = DataLoader(train_tensor, batch_size = batch_size, shuffle=True, num_workers=2)
dev_loader = DataLoader(dev_tensor, batch_size= batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_tensor, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
# function to show an image

def imshow(img):
    npimg = img.numpy()
    npimg = np.transpose(npimg,(2,1,0))
    plt.imshow(npimg, cmap='gray')
    plt.show()

In [ ]:
# get some random training images

dataiter = iter(train_loader)
img1, img2, img3, _map, labels, _ = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(img1[0]))
imshow(torchvision.utils.make_grid(img2[0]))
imshow(torchvision.utils.make_grid(img3[0]))
imshow(torchvision.utils.make_grid(_map[0].unsqueeze(dim=0)))

In [ ]:
class e2dNet(nn.Module):
    def __init__(self):
        super(e2dNet,self).__init__()
        self.center_conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=24, kernel_size = 5, stride = 2),
#             nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.Conv2d(in_channels=24,out_channels=36, kernel_size=5, stride =2),
#             nn.BatchNorm2d(36),
            nn.ReLU(),
            nn.Conv2d(in_channels=36,out_channels=48, kernel_size=3, stride =2),
#             nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(in_channels=48,out_channels=64, kernel_size=3, stride =1),
#             nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Flatten()
        )
        self.left_conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=24, kernel_size = 5, stride = 2),
#             nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.Conv2d(in_channels=24,out_channels=36, kernel_size=5, stride =2),
#             nn.BatchNorm2d(36),
            nn.ReLU(),
            nn.Conv2d(in_channels=36,out_channels=48, kernel_size=3, stride =2),
#             nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(in_channels=48,out_channels=64, kernel_size=3, stride =1),
#             nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Flatten()
        )
        self.right_conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=24, kernel_size = 5, stride = 2),
#             nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.Conv2d(in_channels=24,out_channels=36, kernel_size=5, stride =2),
#             nn.BatchNorm2d(36),
            nn.ReLU(),
            nn.Conv2d(in_channels=36,out_channels=48, kernel_size=3, stride =2),
#             nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(in_channels=48,out_channels=64, kernel_size=3, stride =1),
#             nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Flatten()
        )
        self.map_conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=24, kernel_size = 5, stride = 2),
#             nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.Conv2d(in_channels=24, out_channels=36, kernel_size = 5, stride = 2),
#             nn.BatchNorm2d(36),
            nn.ReLU(),
            nn.Conv2d(in_channels=36, out_channels=48, kernel_size = 3, stride = 2),
#             nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            nn.Linear(in_features=24960, out_features=1000),
            nn.ReLU(),
            nn.Linear(in_features=1000, out_features=100),
            nn.ReLU(),
        )
        self.phi = nn.Sequential(
            nn.Linear(in_features=100,out_features=3),
            nn.Softmax(dim=1)
        )
        self.mean = nn.Linear(in_features=100,out_features=3)
        self.sigma = nn.Sequential(
            nn.Linear(in_features=100,out_features=3),
            nn.Softplus()
        )    
        
        
    def forward(self,x_c,x_l,x_r,m):
        x_c = self.center_conv(x_c)
        x_l = self.left_conv(x_l)
        x_r = self.right_conv(x_r)
        m = self.map_conv(m)
        x_con = torch.cat((x_c,x_l,x_r,m),dim=1)
        x_con = self.fc(x_con)
        phi = self.phi(x_con)
        mean = self.mean(x_con)
        sigma = self.sigma(x_con)
        
        return phi, mean, sigma
        
        

In [ ]:
def Loss_NLL(phi,mean,sigma, data):
    
    p_g = torch.exp(-torch.div((mean-data)**2,(2*sigma**2)))
    p_g = phi*torch.div(p_g,math.sqrt(2*math.pi)*sigma)
#     p_g = phi*p_g

    p_g = torch.sum(p_g,dim=1)
    
    return torch.sum(-torch.log(p_g))/batch_size



#     return torch.min(-torch.log(p_g),
#                -torch.log(torch.tensor(threshold).float().to(device)))/batch_size

#     return torch.sum(phi*(mean-data)**2)

In [ ]:
def p_norm(x, p):
    
    norm = torch.pow(x,p)
    norm = torch.sum(norm,dim=1)
    
    return torch.sum(torch.pow(norm,1/p))/batch_size

In [ ]:
def regularizer(sig, c):
    # lasso
#     psi = torch.abs(torch.log(sig)-c)
    
    # ridge
    psi = (torch.log(sig)-c)**2
    
    
    
    return torch.sum(psi)/batch_size

In [ ]:
### Hyper parameters ###

C_regularizer = -5
epoch = 100

#########################

# initialize model
net = e2dNet()

# Training on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)

# Define a Loss function and optimizer
optimizer = optim.Adam(net.parameters(), lr=1e-6)

In [ ]:
def training(net, trainloader, devloader , max_epoch, C_regularizer, optimizer, model_path='e2e_net_v2.pth'):
    
    dev_loss = 100000000
    
    for epoch in range(max_epoch):
        running_loss = 0
        
        net.train()
        for i, data in enumerate(trainloader):
            
            
            img_c, img_l, img_r, _map, label, _ = data
            
            img_c = img_c.float().to(device)
            img_l = img_l.float().to(device)
            img_r = img_r.float().to(device)
            _map = _map.float().to(device)
            label = label.float().to(device)
            
            optimizer.zero_grad()
            
            phi, mean, sigma = net(img_c,img_l,img_r,_map)
#             print(phi,mean,sigma)
#             loss = Loss_NLL(phi, mean, sigma, (label).unsqueeze(dim=1)*math.pi/180)
            loss = Loss_NLL(phi, mean, sigma, (label).unsqueeze(dim=1))

            loss += 1*p_norm(phi,0.5)
#             loss += torch.sum(torch.norm(phi, 0.5, dim=1))/batch_size
            loss += 0.1*regularizer(sigma, C_regularizer)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i%10 == 9:
                print('[%d, %5d] training loss: %.3f' %
                     (epoch+1, i+1, running_loss/10))
                running_loss = 0
        
        # dev set evaluation
        net.eval()
        running_loss = 0
        count = 0
        for i, data in enumerate(devloader):
            
            img_c, img_l, img_r, _map, label, _ = data
            
            img_c = img_c.float().to(device)
            img_l = img_l.float().to(device)
            img_r = img_r.float().to(device)
            _map = _map.float().to(device)
            label = label.float().to(device)
            
            phi, mean, sigma = net(img_c,img_l,img_r,_map)
            loss = Loss_NLL(phi, mean, sigma, (label).unsqueeze(dim=1))

            loss += 1*p_norm(phi,2)
#             loss += torch.sum(torch.norm(phi, 0.5, dim=1))/batch_size
            loss += 1*regularizer(sigma, C_regularizer)
            
            running_loss += loss.item()
            count = i+1
        print('[%d, %5d] validation loss: %.3f' %
             (epoch+1, i+1, running_loss/count))
        
        if dev_loss > running_loss/count:
            dev_loss = running_loss/count
            print("Training finished")
            torch.save(net.state_dict(), model_path)
            print("Model saved")

In [ ]:
training(net, train_loader, dev_loader, epoch, C_regularizer, optimizer, model_path='e2e_net_v2.4.pth')

####

# e2e_net_v2 : norm2 사용
# e2e_net_v2.1 : norm 0.5 사용
# e2e_net_v2.2 : norm 0.5, 가중치 0.01사용
# e2e_net_v2.3 : norm 2 사용, 가중치 0.01, validation set 추가
# e2e_net_v2.3 : norm 2 사용, 가중치 1, validation set 추가

In [ ]:
from PIL import Image, ImageDraw

def tensor2array(img,mean):
#     print(img.shape)
    im = np.transpose(img.numpy(),(2,1,0))
    
    im = Image.fromarray(im.astype(np.uint8))
    
    return im, mean.numpy()

def draw_path(img, mean, GT):
    img_o, theta = tensor2array(img, mean)
    R = -1/ theta
    if GT==0:
        GT = 100000
    else:
        GT = -1/GT.numpy()
#     print(R)
    
    img_o = draw_help(img_o,R[0],'red')
    img_o = draw_help(img_o,R[1],'red')
    img_o = draw_help(img_o,R[2],'red')
    img_o = draw_help(img_o,GT,'blue')
    
    image_arr = np.asarray(img_o)
    plt.imshow(image_arr)
    plt.show()
    
    

def draw_help(img, R, color):
    h=1.7
    h_t = 1280/720*h
    y0=1.6
    p=360

    u0 = 540
    v0 = 360

    num = 1000

    y = np.linspace(h,min(abs(R-y0)*2/3,50),num)
    if R >= 0 :
        x = R - np.sqrt(R**2 - (y+y0)**2)
    else :
        x = R + np.sqrt(R**2 - (y+y0)**2)  

    u = (p*x*h_t/y/h + u0).astype(np.int32)
    v = (p*h_t/y + v0).astype(np.int32)


    point_list = []
    for i in range(num):
        point_list.append((u[i],v[i]))

    im_path = ImageDraw.Draw(img)
    im_path.point(point_list, fill= color)

    return img


In [ ]:
def test(testloader, model_path='e2e_net.pth'):
    net = e2dNet()
    net.load_state_dict(torch.load(model_path))
    net.eval().to(device)
    
    with torch.no_grad():
        for data in testloader:
            
            img_c, img_l, img_r, _map, label, img_o = data
            data_size = img_c.shape[0]
            
            for num in range(batch_size):
                if num >= data_size:
                    break

                imshow(torchvision.utils.make_grid(img_c[num]))
                imshow(torchvision.utils.make_grid(img_l[num]))
                imshow(torchvision.utils.make_grid(img_r[num]))
                imshow(torchvision.utils.make_grid(_map[num].unsqueeze(dim=0)))

                img_c_t = img_c[num].unsqueeze(dim=0).float().to(device)
                img_l_t = img_l[num].unsqueeze(dim=0).float().to(device)
                img_r_t = img_r[num].unsqueeze(dim=0).float().to(device)
                _map_t = _map[num].unsqueeze(dim=0).float().to(device)
                label_t = label[num].unsqueeze(dim=0).float().to(device)

                phi, mean, sigma = net(img_c_t,img_l_t,img_r_t,_map_t)

                draw_path(img_o[num].float(), mean.squeeze().cpu(), label_t.squeeze().cpu())

                print("result ",phi, mean, sigma)
#                 print(label_t)


In [ ]:
test(test_loader,model_path='e2e_net_v2.1.pth')